In [ ]:
#==============================================================================================
# sbert 모델을 각 4가지 평가 데이터셋으로 하나씩 지정해서 평가하는 예시임
# => 평가 데이터 셋 : Korsts, Kluests, gluests, stsb_multi_mt 
# => 각 개별적 eval 출력 하는 예시는 sbert-test2.ipynb 참조
#=> ** skt/kobert-base-V1  테스트시에는  tokenizer_config.json 에 tokenizer_class:"XLNetTokenizer" 인지 확인해야 함
#==============================================================================================
import os
import sys
import time
from sentence_transformers import models, losses
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.evaluation import SimilarityFunction

sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging

logger = mlogging(loggername="s-bert-test", logfilename="../../log/s-bert-test")
device = GPU_info()
seed_everything(111)

In [ ]:
## 평가 설정 parameter 들 ########################################

# 평가할 bert 모델 혹은 sbert모델 경로
#=> ** skt/kobert-base-V1  테스트시에는  tokenizer_config.json 에 tokenizer_class:"XLNetTokenizer" 인지 확인해야 함
bisSKKobertModel = 0  # skt/kobert-base-V1 허깅페이스 모델 사용시에는 1로 해줌
model_path = "../../data11/model/sbert/bert-base-multilingual-cased-sts-glue/" #distilbert-base-multilingual-cased-sts
#model_path = "kpfbert-sts"


# 유사도 측정방식(COSINE, EUCLIDEAN, MANHATTAN, DOT_PRODUCT 중 선택 , 모두 spearman 방식임)
# => None 이면 아래 값들중 MAX 값 추력함
#main_similarity = None
main_similarity = SimilarityFunction.COSINE
#main_similarity = SimilarityFunction.EUCLIDEAN
#main_similarity = SimilarityFunction.MANHATTAN
#main_similarity = SimilarityFunction.DOT_PRODUCT

# 평가시 cosine 유사도등 측정 결과값 파일 (similarity_evaluation_xxxx.xls) 저장될 경로
output_path = 'eval'
os.makedirs(output_path, exist_ok=True)
csvfilename = 'V3.1.1'   # 유사도 저장할 파일명 (eval/similarity_evaluation_{csffilename}.csv 형식으로 저장)

# 평가 sts 형태의 말뭉치 파일들
use_korsts = 0     #  korsts 파일
use_kluests = 0    # kluests_v1.1 파일
use_glue_sts = 1   # true이면 영문 glue_sts 데이터셋 추가하여 평기 시킴
use_en_sts = 0     # true이면 영문 stsb_multi_mt데이터셋 추가하여 평가시킴.

# * 모델에 상관없이 영어는 소문자로 입력하겠다는 뜻(glue는 소문자로만 비교하는게 더 적확함), 한국어는 True도 상관없음
do_lower_case = False
'''
if use_glue_sts == 1 or use_en_sts == 1:
    do_lower_case = True 
else:
    do_lower_case = False
'''    
# korsts tsv 파일 경로
korsts_file = '../../data11/korpora/korsts/tune_test.tsv'

# kluests json 파일 경로
kluests_file = '../../data11/korpora/klue-sts/klue-sts-v1.1_dev.json'

# glue test tsv 파일경로
glue_test_file = '../../data11/korpora/gluestsb/gluestsb-test.tsv'

eval_batch_size = 64 # 배치파일사이즈 = 크면 eval 시간이 단축됨.단 gpu 메모리 올라감.
max_seq_length = 72

In [ ]:
from datasets import load_dataset
test_samples = []


# /korsts/tune_test.tsv 파일을 불러옴
if use_korsts == True:
    with open(korsts_file, 'rt', encoding='utf-8') as fIn1:
        lines = fIn1.readlines()
        for line in lines:
            s1, s2, score = line.split('\t')
            score = score.strip()
            score = float(score) / 5.0
            test_samples.append(InputExample(texts=[s1,s2], label=score))


# /klue-sts/klue-sts-v1.1_dev.json 파일을 불러옴
if use_kluests == True:
    import json

    with open(kluests_file, "r", encoding='utf-8') as fIn2:
        data = json.load(fIn2)
        for el in data:
            s1 = el["sentence1"]
            s2 = el["sentence2"]
            score = el["labels"]['label']
            test_samples.append(InputExample(texts=[s1,s2], label=score))
        
# 영문 stsb_multi_mt test 버전 파일 로딩함
if use_en_sts == True:
    en_sts_dataset = load_dataset("stsb_multi_mt", name="en", split="test")
    for data in en_sts_dataset:
        text_a = data["sentence1"]
        text_b = data["sentence2"]
        score = data["similarity_score"]
        score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
        test_samples.append(InputExample(texts= [text_a,text_b], label=score))
        
if use_glue_sts == True: 
    
    #glue stsb valildation 데이터셋 불러옴(subset : "stsb" = 1,500)
    glue_stsb_dataset = load_dataset("glue","stsb", split="validation")
    for data in glue_stsb_dataset:
        text_a = data["sentence1"]
        text_b = data["sentence2"]
        score = data["label"]
        score = float(score) / 5.0  #5로 나눠서 0~1 사이가 되도록 함
        test_samples.append(InputExample(texts= [text_a,text_b], label=score))
    '''
    # glue-test는 허깅페이스 데이터셋에는 labe(score)가 -1로 등록되어 있어서, 실제 glue 사이트에서 org파일을 받아서 불러옴
    # => 해당 sts-test.tsv 파일에서 몇가지 주석달린 문장들은 제거함(약 300개)
    with open(glue_test_file, 'rt', encoding='utf-8') as fIn1:
        lines = fIn1.readlines()
        for idx, line in enumerate(lines):
            if idx == 0:
                continue
            _, _, _, _, score, text_a, text_b = line.split('\t')
            score = score.strip()
            score = float(score) / 5.0
            test_samples.append(InputExample(texts= [text_a,text_b], label=score))
    '''

print(test_samples[0:3])
print(f'*test_samples_len: {len(test_samples)}')

In [ ]:
# 모델과 tokenizer 를 불러옴
# => **사전파일(vocab.txt, *.json) 와 model 경로(config.json, pytorch_model.bin)가 같은 경로에 있어야 함.
word_embedding_model = models.Transformer(model_path, max_seq_length=max_seq_length, do_lower_case=do_lower_case)

#========================================================================================================
# skt/kobert 모델은 tokenizer을 XLNET Tokenizer 이므로, 자체 KoBERTTOkenizer 를 불러와서 사용해야 함.
# => 설치 : !pip install 'git+https://github.com/SKTBrain/KOBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# => 출처 : https://velog.io/@m0oon0/KoBERT-%EC%82%AC%EC%9A%A9%EB%B2%95
if bisSKKobertModel == 1:
    from kobert_tokenizer import KoBERTTokenizer
    word_embedding_model.tokenizer = KoBERTTokenizer.from_pretrained(model_path)
#========================================================================================================

# embedding 길이를 재조정 필요할때 auto_model.resize_token_embeddings 해줌
print(f'token_len:{len(word_embedding_model.tokenizer)}')
#word_embedding_model.auto_model.resize_token_embeddings(len(word_embedding_model.tokenizer))

# 2 bert 모델의 임베딩 풀링 정책을 설정(cls 이용, 워드임베딩 평균이용, 워드임베딩 max 이용)
# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),  #모델이 dimension(768)
                               pooling_mode_mean_tokens=True,  # 워드 임베딩 평균을 이용
                               pooling_mode_cls_token=False,   # cls 를 이용
                               pooling_mode_max_tokens=False)  # 워드 임베딩 값중 max 값을 이용

logger.info("\n")
logger.info("======================TEST===================")

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
print(model)

#model = SentenceTransformer(smodel_path, device='cpu')
#model = SentenceTransformer(smodel_path, cache_folder=output_path)
#model.to(device)
logger.info(f'{model}')

In [ ]:
logger.info(f"*main_similarity: {main_similarity}")
logger.info(f"*model path: {model_path}")

if use_korsts == True:
    logger.info(f"*korsts file({korsts_file})")

if use_kluests == True:
    logger.info(f"*klue file({kluests_file})")
    
if use_en_sts == True:
    logger.info(f"*stsb_multi_mt")
  
if use_glue_sts == True:
    logger.info(f"*glue(stsb)")
    
start = time.time()

test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, main_similarity=main_similarity, batch_size=eval_batch_size, name=csvfilename, show_progress_bar=True)
result = test_evaluator(model, output_path=output_path)
logger.info(f"\n")
        
logger.info(f'=== result: {result} ===')
logger.info(f'=== 처리시간: {time.time() - start:.3f} 초 ===')
logger.info("=====================================================")
logger.info("\n")
